# Install transformers 2.8.0

In [2]:
!pip install transformers==2.8.0

     |████████████████████████████████| 573kB 8.9MB/s 
     |████████████████████████████████| 3.7MB 24.3MB/s 
     |████████████████████████████████| 890kB 30.6MB/s 
     |████████████████████████████████| 1.1MB 52.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=25bc4a4bbb54b5d448515d3a73b3760ab7a7378b5e3e8ad7696c6fde6d70380e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [1]:
!nvidia-smi

Fri May 15 06:55:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Set working directory to the directory containing `download_glue_data.py` and `run_glue.py`

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!pwd
import os
os.chdir('/content/drive/My Drive/NLU project')
!pwd

/content
/content/drive/My Drive/NLU project


# Download GLUE data

In [0]:
GLUE_DIR="data/glue"

In [6]:
!echo $GLUE_DIR

data/glue


In [0]:
TASK_NAME="CoLA"

In [8]:
!echo $TASK_NAME

CoLA


In [9]:
!python download_glue_data.py --help

usage: download_glue_data.py [-h] [--data_dir DATA_DIR] [--tasks TASKS]
                             [--path_to_mrpc PATH_TO_MRPC]

optional arguments:
  -h, --help            show this help message and exit
  --data_dir DATA_DIR   directory to save data to
  --tasks TASKS         tasks to download data for as a comma separated string
  --path_to_mrpc PATH_TO_MRPC
                        path to directory containing extracted MRPC data,
                        msr_paraphrase_train.txt and msr_paraphrase_text.txt


In [10]:
!python download_glue_data.py --data_dir $GLUE_DIR --tasks $TASK_NAME

	Completed!


# Compute score for specific GLUE task

In [11]:
MODEL_DIR = os.path.join('models', 'roberta_token_discrimination')
os.path.exists(MODEL_DIR)

True

In [0]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [0]:
!python run_glue.py --help

2020-05-15 06:02:05.223176: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
usage: run_glue.py [-h] --data_dir DATA_DIR --model_type MODEL_TYPE
                   --model_name_or_path MODEL_NAME_OR_PATH --task_name
                   TASK_NAME --output_dir OUTPUT_DIR
                   [--config_name CONFIG_NAME]
                   [--tokenizer_name TOKENIZER_NAME] [--cache_dir CACHE_DIR]
                   [--max_seq_length MAX_SEQ_LENGTH] [--do_train] [--do_eval]
                   [--evaluate_during_training] [--do_lower_case]
                   [--per_gpu_train_batch_size PER_GPU_TRAIN_BATCH_SIZE]
                   [--per_gpu_eval_batch_size PER_GPU_EVAL_BATCH_SIZE]
                   [--gradient_accumulation_steps GRADIENT_ACCUMULATION_STEPS]
                   [--learning_rate LEARNING_RATE]
                   [--weight_decay WEIGHT_DECAY] [--adam_epsilon ADAM_EPSILON]
                   [--max_grad_norm MAX_G

In [12]:
!python run_glue.py \
    --model_type roberta \
    --model_name_or_path $MODEL_DIR \
    --task_name $TASK_NAME \
    --do_train \
    --do_eval \
    --data_dir $GLUE_DIR/$TASK_NAME \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=64   \
    --per_gpu_train_batch_size=64   \
    --learning_rate 2e-5 \
    --num_train_epochs 3 \
    --output_dir run \
    --overwrite_output_dir

2020-05-15 06:56:30.340154: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
05/15/2020 06:56:32 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
05/15/2020 06:56:32 - INFO - transformers.configuration_utils -   loading configuration file models/roberta_token_discrimination/config.json
05/15/2020 06:56:32 - INFO - transformers.configuration_utils -   Model config RobertaConfig {
  "_num_labels": 2,
  "architectures": [
    "RobertaForTokenDiscrimination"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": 0,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "finetuning_task": "cola",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermedia